# Valley Bottom Extraction
## Iterate over regional outputs and select input datasets for RVBD tool as well as convert/create necessary datasets.

Create data dictionary to store input parameters for tool.
1. River Network
2. Catchments
3. DEM
4. Wetlands - as feature class or shapefile
5. Output Folder - Directory
6. River Valley Bottom - Output River Valley Bottom Polygon

In [6]:
# Convert wetlands to shapefile
import arcpy
import os
import sys
print('imports complete')

imports complete


In [4]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [7]:
# folder containing regional watersheds and covariate source data
datadir = r'D:\\GIS\\AKSSF'
arcpy.env.overwriteOutput = True
# Set data dir equal to directory containing the AKSSF regional sub-folders.
data_dir = r"D:\\GIS\\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
# Regions to be processed
rList = ['Bristol_Bay','Cook_Inlet', 'Copper_River', 'Prince_William_Sound','Kodiak']
regions = arcpy.ListWorkspaces(workspace_type="Folder")
regions = [os.path.normpath(r) for r in regions if os.path.basename(r) in rList]
#working gdb to save intermediate outputs
scratchgdb = r"D:\\GIS\\scratch.gdb"
scratchfol = r"D:\\GIS\\scratch"
print (regions)


['D:\\GIS\\AKSSF\\Bristol_Bay', 'D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River', 'D:\\GIS\\AKSSF\\Kodiak', 'D:\\GIS\\AKSSF\\Prince_William_Sound']


### Create data dictionary to use as input to RVBD script.  
 * First run using Kodiak as a test worked well with some minor edits but there are issues with the valley bottoms in larger waterbodies.  Maybe just select and merge or append to dataset after running or include in the wetlands layer?  

In [ ]:
rvbdDict = {}
# Walk through gdbs and set input data sets. Convert wetland raster to shapefile if this has not already been done.
for region in regions:
    rname = os.path.basename(region)
    print(rname)
    arcpy.env.workspace = region
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    outgdb = gdb[0]
    walk = arcpy.da.Walk(region, datatype=['RasterDataset','FeatureClass'])
    scratchname = os.path.
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            if 'streams_merge' == filename:
                strsource = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,strsource)
            # Set merged watersheds dataset
            elif 'wtds_merge' == filename:
                wtdsource = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,wtdsource)
            elif 'elev_tif' == filename:
                elev_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,elev_path)
            # Grab wetland raster
            elif filename == 'wetlands.tif':
                print(f'wetlands fc will be saved to {outgdb}')
                print(f'{dirpath},{filename}')
                wras_path = os.path.join(dirpath, filename)
                wrasname = filename[:-4]
                wetfcpath = os.path.join(outgdb, wrasname)
                wetfcpath2 = os.path.join(scratchgdb, wrasname)
                # Make local copy projected in AKAlbers
                if not arcpy.Exists(wetfcpath):
                    print(f'Converting {wrasname} to feature class...')
                    print('----------')
                    wetfc_conv = arcpy.RasterToPolygon_conversion(wras_path,wetfcpath2,raster_field='VALUE',
                                                             simplify='NO_SIMPLIFY',
                                                             create_multipart_features='MULTIPLE_OUTER_PART')
                    wetfc = arcpy.FeatureClassToFeatureClass_conversion(wetfcpath2,outgdb,wrasname,
                                                                        where_clause='gridcode = 1')
                else:
                    print(f'{wetfcpath} already created')
                    wetfc = wetfcpath
                    print('----------')
                append_value(rvbdDict,rname,wetfcpath)
                if not arcpy.exists(os.path.join(dirname,))
                append_value(rvbdDict,rname,scratchfol)
                rivbottpath = os.path.join(dirpath + '.gdb','riv_valbot')
                append_value(rvbdDict,rname,rivbottpath)

In [9]:
#Check paths in dictionary
rvbdDict

{'Bristol_Bay': ['D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wetlands',
  'D:\\\\GIS\\\\scratch',
  'D:\\GIS\\AKSSF\\Bristol_Bay.gdb\\riv_valbot',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\streams_merge',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wtds_merge'],
 'Cook_Inlet': ['D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\wetlands',
  'D:\\\\GIS\\\\scratch',
  'D:\\GIS\\AKSSF\\Cook_Inlet.gdb\\riv_valbot',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\wtds_merge'],
 'Copper_River': ['D:\\GIS\\AKSSF\\Copper_River\\Copper_River.gdb\\wetlands',
  'D:\\\\GIS\\\\scratch',
  'D:\\GIS\\AKSSF\\Copper_River.gdb\\riv_valbot',
  'D:\\GIS\\AKSSF\\Copper_River\\Copper_River.gdb\\wtds_merge'],
 'Kodiak': ['D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wetlands',
  'D:\\\\GIS\\\\scratch',
  'D:\\GIS\\AKSSF\\Kodiak.gdb\\riv_valbot',
  'D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\streams_merge',
  'D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wtds_merge'],
 'Prince_William_Sound': ['D:\\GIS\\AKSSF\\Prince_William_

In [11]:

#Begin River Valley Bottom tool
#Path to folder containing river valley bottom scripts
rvbd_dir = r"D:\\RVBD\\"
#Add RVBD to sys paths
sys.path.append(rvbd_dir)

# Import modules
import arcpy
import os
from pathlib import Path
import datetime
from slope import slope
from valleycostdistance import valley
from costdistance import costdist



In [ ]:
#Create region specific scratch gdb and folders in main RVB_scratch folder
###CODE##

In [12]:
# need to iterate over regions and inputs from data dictionary
# Starting script
start = datetime.datetime.now()
print("Script started at %s" % start)
arcpy.AddMessage ("Script started started at %s" % start)
arcpy.AddMessage ("\n")

# Check out spatial extention
arcpy.CheckOutExtension("Spatial")

# Environments
arcpy.env.overwriteOutput = True

#try with test data
# Python inputs
riv =  r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\streams_merge" # river network as a shapefile
cat = r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wtds_merge" # catchments as a shapefile
dem = r"D:\\GIS\\AKSSF\\Kodiak\\elev.tif"# elevation model as a raster
wet = r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wetlands" # measured valley bottom, wetlands or any wet signature in low areas
pth = r"D:\\GIS\\RVBD_scratch\\Kodiak_RVBD" # path to outputs (scratch and cost distance folders)
outval = r"D:\\GIS\\RVBD_scratch\\Kodiak_RVBD\\Kodiak_RVBD.gdb\\kod_rvbd" # path to output river valley
slp_in = r"D:\\GIS\\AKSSF\\Kodiak\\slope.tif"

# # ArcGIS inputs
# riv =  arcpy.GetParameterAsText(0) # river network as a shapefile
# cat = arcpy.GetParameterAsText(1) # catchments as a shapefile
# dem = arcpy.GetParameterAsText(2) # elevation model as a raster
# wet = arcpy.GetParameterAsText(3) # measured river valley
# pth = arcpy.GetParameterAsText(4) # path to outputs (scratch and cost distance folders)
# outval = arcpy.GetParameterAsText(5) # path to output river valley

# Check to see if scratch geodatabase exists
s = Path(os.path.join(pth, "outputs.gdb"))
if s.exists() == False:
    scrg = arcpy.CreateFileGDB_management(pth, "outputs")
    scr = scrg.getOutput(0) # scratch folder
else:
    scr = os.path.join(pth, "outputs.gdb") #scr = r"in_memory" (if you want to store in memory)

# Check to see if cost dist folder exists (use code in comments if you want results in a folder instead of gdb)
c = Path(os.path.join(pth, "costdist.gdb")) # Path(os.path.join(pth, "costdist"))
if c.exists() == False:
    cstg = arcpy.CreateFileGDB_management(pth, "costdist")  # arcpy.CreateFolder_management(pth, "costdist")
    cst = cstg.getOutput(0) # cost distance folder
else:
    cst = os.path.join(pth, "costdist.gdb") # os.path.join(pth, "costdist")

# # Calculate slope and replace 0 slopes with small value - We already have slope rasters so no need for this step
# slope(riv, cat, dem, scr)
#
# Cost distance calculation
costdist(riv, cat, slp_in, cst, scr)

# Extract valley
valley(riv, wet, cst, scr, outval)


# Ending script
end = datetime.datetime.now()
print("Script ended at %s" % end)
arcpy.AddMessage ("Script ended at %s" % end)
arcpy.AddMessage ("\n")
time_elapsed = end - start
print("Time elapsed %s" % (time_elapsed))
arcpy.AddMessage ("Time elapsed %s" % time_elapsed)
arcpy.AddMessage ("\n")
print(".........................................................")
arcpy.AddMessage (".........................................................")
arcpy.AddMessage ("\n")


Script started at 2022-01-04 12:47:15.727637
Cost dist calculation started at 2022-01-04 12:47:15.730636
There is/are 28 intersecting features
Calculating cost distance for OID 1 at 2022-01-04 12:47:51.379217
Completed cost distance for OID 1 at 2022-01-04 12:48:36.821666
.........................................................
Calculating cost distance for OID 2 at 2022-01-04 12:48:36.825695
Completed cost distance for OID 2 at 2022-01-04 12:49:16.325969
.........................................................
Calculating cost distance for OID 3 at 2022-01-04 12:49:16.327967
Completed cost distance for OID 3 at 2022-01-04 12:49:48.559649
.........................................................
Calculating cost distance for OID 4 at 2022-01-04 12:49:48.561648
Completed cost distance for OID 4 at 2022-01-04 12:50:27.492292
.........................................................
Calculating cost distance for OID 5 at 2022-01-04 12:50:27.493291
Completed cost distance for OID 5 at 20